In [97]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
import requests
import json
import pprint
import time

import pandas as pd

# fix ssl certificate (needed for MacOS sometimes)
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [27]:
CAP_FRIENDLY_BASE_URL = 'https://www.capfriendly.com'

def get_soup(url):
    page = urlopen(url)
    html = page.read().decode("utf-8")
    soup = BeautifulSoup(html, 'html.parser')
    return soup

In [28]:
# get links to each team page
home_page_soup = get_soup(CAP_FRIENDLY_BASE_URL)

teams = {}
for team in home_page_soup.find('div', {'class': 'mb10'}).find_all('a', {'class': 'whi'}):
    teams[team.get_text()] = team.get('href')
pprint.pprint(teams)

{'Anaheim Ducks': '/teams/ducks',
 'Arizona Coyotes': '/teams/coyotes',
 'Boston Bruins': '/teams/bruins',
 'Buffalo Sabres': '/teams/sabres',
 'Calgary Flames': '/teams/flames',
 'Carolina Hurricanes': '/teams/hurricanes',
 'Chicago Blackhawks': '/teams/blackhawks',
 'Colorado Avalanche': '/teams/avalanche',
 'Columbus Blue Jackets': '/teams/bluejackets',
 'Dallas Stars': '/teams/stars',
 'Detroit Red Wings': '/teams/redwings',
 'Edmonton Oilers': '/teams/oilers',
 'Florida Panthers': '/teams/panthers',
 'Los Angeles Kings': '/teams/kings',
 'Minnesota Wild': '/teams/wild',
 'Montreal Canadiens': '/teams/canadiens',
 'Nashville Predators': '/teams/predators',
 'New Jersey Devils': '/teams/devils',
 'New York Islanders': '/teams/islanders',
 'New York Rangers': '/teams/rangers',
 'Ottawa Senators': '/teams/senators',
 'Philadelphia Flyers': '/teams/flyers',
 'Pittsburgh Penguins': '/teams/penguins',
 'San Jose Sharks': '/teams/sharks',
 'Seattle Kraken': '/teams/kraken',
 'St. Louis Bl

In [52]:
# scrape all staff members and get links to their pages
staffs = {}
for team, link in teams.items():
    team_name_url = link.split('/')[2]
    staff_link = f'{CAP_FRIENDLY_BASE_URL}/teams/staff/{team_name_url}'
    print(staff_link)

    staff_page_soup = get_soup(staff_link)
    for staff in staff_page_soup.find('tbody').find_all('tr'):
        for data in staff.find_all('a'):
            staffs[data.get_text()] = data.get('href')

https://www.capfriendly.com/teams/staff/ducks
https://www.capfriendly.com/teams/staff/flames
https://www.capfriendly.com/teams/staff/oilers
https://www.capfriendly.com/teams/staff/kings
https://www.capfriendly.com/teams/staff/sharks
https://www.capfriendly.com/teams/staff/kraken
https://www.capfriendly.com/teams/staff/canucks
https://www.capfriendly.com/teams/staff/goldenknights
https://www.capfriendly.com/teams/staff/coyotes
https://www.capfriendly.com/teams/staff/blackhawks
https://www.capfriendly.com/teams/staff/avalanche
https://www.capfriendly.com/teams/staff/stars
https://www.capfriendly.com/teams/staff/wild
https://www.capfriendly.com/teams/staff/predators
https://www.capfriendly.com/teams/staff/blues
https://www.capfriendly.com/teams/staff/jets
https://www.capfriendly.com/teams/staff/hurricanes
https://www.capfriendly.com/teams/staff/bluejackets
https://www.capfriendly.com/teams/staff/devils
https://www.capfriendly.com/teams/staff/islanders
https://www.capfriendly.com/teams/sta

In [54]:
len(staffs)

541

In [99]:
all_signings = []
for staff_member, link in staffs.items():
    # scrape
    staff_member_soup = get_soup(CAP_FRIENDLY_BASE_URL + link)

    # extract data
    signigns_for_this_staff_member = []
    name = staff_member_soup.find('h1').get_text()
    try:
        for signing in staff_member_soup.find('div', {'id': 'staff_signings_area'}).find('tbody').find_all('tr'):
            signing_data = []
            for td in signing.find_all('td'):
                signing_data.append(td.get_text())
            for a in signing.find_all('a'):
                signing_data.append(a.get('href'))
            signing_data.append(name) # add GM name
            signigns_for_this_staff_member.append(signing_data)
            
        print(f'found {len(signigns_for_this_staff_member)} signings for staff member {name}')
        all_signings.extend(signigns_for_this_staff_member)
    except Exception:
        # not a gm, do nothing
        print(f'staff member {name} has zero signings as GM')
    time.sleep(0.25) # sleep to avoid api limits

staff member FRANCOIS BEAUCHEMIN has zero signings as GM
staff member NEWELL BROWN has zero signings as GM
staff member GREG CRONIN has zero signings as GM
staff member ROB DIMAIO has zero signings as GM
staff member BRUCE FRANKLIN has zero signings as GM
staff member RYAN GETZLAF has zero signings as GM
staff member CRAIG JOHNSON has zero signings as GM
staff member MARTIN MADDEN JR has zero signings as GM
staff member SUDARSHAN MAHARAJ has zero signings as GM
staff member SCOTT NIEDERMAYER has zero signings as GM
staff member RICK PATERSON has zero signings as GM
staff member RICH PRESTON has zero signings as GM
staff member TIM RYAN has zero signings as GM
staff member SUSAN SAMUELI has zero signings as GM
staff member HENRY SAMUELI has zero signings as GM
staff member MICHAEL SCHULMAN has zero signings as GM
found 0 signings for staff member JEFF SOLOMON
staff member MIKE STOTHERS has zero signings as GM
staff member BRENT THOMPSON has zero signings as GM
found 41 signings for staf

In [109]:
df = pd.DataFrame(all_signings)
df.columns = ['playerName', 'teamName', 'signDate', 'contractType', 'term', 'aav', 'totalValue', 'playerLink', 'teamLink', 'generalManager']
df.to_csv('contracts.csv', index=False)
df.head(10)

,playerName,teamName,signDate,contractType,term,aav,totalValue,playerLink,teamLink,generalManager
0,"Klingberg, John",Anaheim Ducks,"Jul. 29, 2022",Standard (UFA),1 year,"$7,000,000","$7,000,000",/players/john-klingberg,/teams/ducks,PAT VERBEEK
1,"Terry, Troy",Anaheim Ducks,"Aug. 2, 2023",Standard (RFA),7 years,"$7,000,000","$49,000,000",/players/troy-terry,/teams/ducks,PAT VERBEEK
2,"Killorn, Alex",Anaheim Ducks,"Jul. 1, 2023",Standard (UFA),4 years,"$6,250,000","$25,000,000",/players/alex-killorn,/teams/ducks,PAT VERBEEK
3,"Zegras, Trevor",Anaheim Ducks,"Oct. 2, 2023",Standard (RFA),3 years,"$5,750,000","$17,250,000",/players/trevor-zegras,/teams/ducks,PAT VERBEEK
4,"Strome, Ryan",Anaheim Ducks,"Jul. 13, 2022",Standard (UFA),5 years,"$5,000,000","$25,000,000",/players/ryan-strome,/teams/ducks,PAT VERBEEK
5,"Gudas, Radko",Anaheim Ducks,"Jul. 1, 2023",Standard (UFA),3 years,"$4,000,000","$12,000,000",/players/radko-gudas,/teams/ducks,PAT VERBEEK
6,"Vatrano, Frank",Anaheim Ducks,"Jul. 13, 2022",Standard (UFA),3 years,"$3,650,000","$10,950,000",/players/frank-vatrano,/teams/ducks,PAT VERBEEK
7,"Drysdale, Jamie",Anaheim Ducks,"Oct. 5, 2023",Standard (RFA),3 years,"$2,300,000","$6,900,000",/players/jamie-drysdale,/teams/ducks,PAT VERBEEK
8,"Lundeström, Isac",Anaheim Ducks,"Jul. 25, 2022",Standard (RFA),2 years,"$1,800,000","$3,600,000",/players/isac-lundestrom,/teams/ducks,PAT VERBEEK
9,"Carlsson, Leo",Anaheim Ducks,"Jul. 12, 2023",Entry-Level (ELC),3 years,"$950,000","$12,600,000",/players/leo-carlsson,/teams/ducks,PAT VERBEEK


------------------

In [111]:
# scrape salary cap history
url = 'https://www.capfriendly.com/salary-cap'
salary_cap_soup = get_soup(url)

In [124]:
salary_cap_history = []
for tr in salary_cap_soup.find('div', {'id': 'salaryCapHistoryInnerContainer'}).find('tbody').find_all('tr'):
    this_year_salary_cap = []
    for td in tr.find_all('td'):
        this_year_salary_cap.append(td.get_text())
    salary_cap_history.append(this_year_salary_cap)

In [128]:
df_cap = pd.DataFrame(salary_cap_history)
df_cap.columns = ['season', 'dateConfirmed', 'percentChange', 'upperLimit', 'lowerLimit', 'minSalary']
df_cap.to_csv('capHistory.csv', index=False)
df_cap

,season,dateConfirmed,percentChange,upperLimit,lowerLimit,minSalary
0,2025-26,NHL Estimate,5.14%,"$92,000,000","$68,000,000","$775,000"
1,2024-25,NHL Estimate,4.79%,"$87,500,000","$64,700,000","$775,000"
2,2023-24,"Jun. 23, 2023",1.21%,"$83,500,000","$61,700,000","$775,000"
3,2022-23,"Mar. 29, 2022",1.23%,"$82,500,000","$61,000,000","$750,000"
4,2021-22,"Jul. 1, 2021",0.00%,"$81,500,000","$60,200,000","$750,000"
5,2020-21,"Jul. 10, 2020",0.00%,"$81,500,000","$60,200,000","$700,000"
6,2019-20,"Jun. 22, 2019",2.52%,"$81,500,000","$60,200,000","$700,000"
7,2018-19,"Jun. 21, 2018",6.00%,"$79,500,000","$58,800,000","$650,000"
8,2017-18,"Jun. 18, 2017",2.74%,"$75,000,000","$55,400,000","$650,000"
9,2016-17,"Jun. 21, 2016",2.24%,"$73,000,000","$54,000,000","$575,000"
